In [1]:
import numpy as np
import simpy
import random
import matplotlib
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [3]:
phase_1_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [4]:
class Job(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = self.env.process(self.call())
        self.run = run_
        
    
    def call(self):
        print('%s initiated at %g' % (self.name, self.env.now))
        self.run.num_jobs_c += 1
        self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
        self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.progress_job())
            print('%s is done at %g' % (self.name, self.env.now))
            self.departure_t = self.env.now
            self.run.num_jobs_c -= 1
            self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
            self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])

            
    def progress_job(self):
        duration = random.expovariate(self.run.serv_rate)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        self.departure_t = self.env.now
        

In [5]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        self.run = False
        self.arr_rate = arr_rate
        self.serv_rate = 1/serv_rate
        self.num_jobs_c = 0
        self.num_jobs_system = [[0,0,-1]]
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = NUM_SERVER)
    
        


    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        self.run = True
        self.average_num_jobs_system()
        
    def show_run(self):
        if self.run!=True:
            self.run_sim()
        
    def average_num_jobs_system(self):
        weighted_value =0
        for arr in self.num_jobs_system:
            if(arr[2]==-1):
                end_of_sim = arr[1]
                break
            weighted_value += arr[0]*(arr[2]-arr[1])
        self.avg_jobs = weighted_value / end_of_sim

    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Job('Job %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [6]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = 1/((lib["arr_interval"][0]+lib["arr_interval"][1])/2)
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)
        self.num_runs = num_runs

    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
            
        print("total run :",len(self.runs)," ")
    
    def create_df(self,util):
        for i in range(len(self.ens_avg[util])):
            if(i == 0):
                ens_sum_avg = [self.ens_avg[util][0]]
                ens_cum_avg = [self.ens_avg[util][0]]
            else:
                ens_sum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]+self.ens_avg[util][i])) 
                ens_cum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]/(i+1)))
        df_arr_avg = pd.DataFrame(self.arr_avg[util],columns=["ARRIVAL"])
        df_ens_avg = pd.DataFrame(self.ens_avg[util],columns=["AVG"])
        df_ens_sum_avg = pd.DataFrame(ens_sum_avg,columns=["SUM-AVG"])
        df_ens_cum_avg = pd.DataFrame(ens_cum_avg,columns=["CUM-AVG"])
        df = pd.concat([df_arr_avg,df_ens_avg,df_ens_sum_avg,df_ens_cum_avg],ignore_index=True,axis=1)

        selected_runs = self.runs[(util)*self.num_runs:(util+1)*self.num_runs]
        
        for run_no, run in enumerate(selected_runs):
            df_time_spend = pd.DataFrame(np.sum([run.service_times,run.queue_w_times],axis=0),columns=["REPLICATION-"+str(run_no)])
            
            print(df_time_spend)
            df = pd.concat([df,df_time_spend],ignore_index=True,axis=1)
        
        return df

    def arrival_averages(self):
        self.arr_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for util_no in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.arr_avg[util_no][i] = self.arr_avg[util_no][i] + self.runs[util_no*self.num_runs+run_no].jobs[i].arrival_t
        self.arr_avg = np.divide(self.arr_avg,self.num_runs)

    def ensemble_averages(self):
        self.ens_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for k in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.ens_avg[k][i] = self.ens_avg[k][i] + (self.runs[k*self.num_runs+run_no].jobs[i].departure_t - self.runs[k*self.num_runs+run_no].jobs[i].arrival_t)
        self.ens_avg = np.divide(self.ens_avg,self.num_runs)
            

In [7]:
phase_1 = Phase(phase_1_lib,10)

In [8]:
phase_1.run_phase()

Job 1 initiated at 6.30123
Job 1 is assigned to an operator at 6.30123
Job 2 initiated at 6.45008
Job 2 is assigned to an operator at 6.45008
Job 3 initiated at 9.73294
Job 1 is done at 12.5274
Job 3 is assigned to an operator at 12.5274
Job 4 initiated at 13.8782
Job 2 is done at 14.0568
Job 4 is assigned to an operator at 14.0568
Job 5 initiated at 14.5975
Job 6 initiated at 15.1339
Job 4 is done at 16.0432
Job 5 is assigned to an operator at 16.0432
Job 3 is done at 16.4762
Job 6 is assigned to an operator at 16.4762
Job 5 is done at 18.9847
Job 6 is done at 19.7573
Job 7 initiated at 26.5302
Job 7 is assigned to an operator at 26.5302
Job 7 is done at 35.8308
Job 8 initiated at 61.686
Job 8 is assigned to an operator at 61.686
Job 9 initiated at 63.6848
Job 9 is assigned to an operator at 63.6848
Job 10 initiated at 67.2551
Job 9 is done at 67.9931
Job 10 is assigned to an operator at 67.9931
Job 11 initiated at 68.3324
Job 10 is done at 68.4237
Job 11 is assigned to an operator at

In [9]:
phase_1.ensemble_averages()
phase_1.arrival_averages()

In [10]:
phase_1.ensemble_averages

<bound method Phase.ensemble_averages of <__main__.Phase object at 0x127bf4650>>

In [11]:
df_0 = phase_1.create_df(0)
df_1 = phase_1.create_df(1)
df_2 = phase_1.create_df(2)
df_3 = phase_1.create_df(3)
df_0.to_csv("df_0.6.csv")
df_1.to_csv("df_0.7.csv")
df_2.to_csv("df_0.8.csv")
df_3.to_csv("df_0.9.csv")

    REPLICATION-0
0        6.226186
1        7.606729
2        4.780905
3        4.127462
4        4.387183
..            ...
95       5.624919
96      13.870891
97       2.446022
98       0.662497
99      16.371818

[100 rows x 1 columns]
    REPLICATION-1
0        5.107829
1        2.718583
2       16.963713
3        0.471642
4        6.024371
..            ...
95      15.739543
96      38.393060
97      10.140299
98      15.016489
99      35.221244

[100 rows x 1 columns]
    REPLICATION-2
0        0.001700
1        4.076066
2        4.089464
3        3.799613
4        3.962883
..            ...
95       9.005735
96      19.657668
97       0.089034
98       4.013842
99       4.229586

[100 rows x 1 columns]
    REPLICATION-3
0        5.148248
1        0.179118
2        2.868680
3        3.073483
4       12.632056
..            ...
95      15.119525
96      11.929440
97      11.763846
98       5.234917
99      10.224845

[100 rows x 1 columns]
    REPLICATION-4
0        9.395827
1   